In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [ ]:
dataset_path=r"/content/drive/MyDrive/Deep_L/Data_1.json"

In [ ]:
def load_data(dataset_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(dataset_path, "r") as fp:
        data = json.load(fp)
    X = np.array(data["mfccs"])
    y = np.array(data["labels"])
    return X, y

In [ ]:
def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.

    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split

    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(dataset_path)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [ ]:
def predict(model, X, y):
    """Predict a single sample using the trained model

    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] # array shape (1, 130, 13, 1)

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)

    print("Target: {}, Predicted label: {}".format(y, predicted_index))

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.initializers import Orthogonal

def build_rnn_model(input_shape):
    model = Sequential([
        # Supprimez la dimension supplémentaire
        layers.Reshape((input_shape[0], input_shape[1])),
        # Couche RNN bidirectionnelle avec GRU
        layers.Bidirectional(layers.GRU(64, return_sequences=True, kernel_initializer=Orthogonal())),
        # Couche Dropout pour éviter le surapprentissage
        layers.Dropout(0.3),
        # Couche RNN bidirectionnelle avec GRU
        layers.Bidirectional(layers.GRU(64, kernel_initializer=Orthogonal())),
        # Couche entièrement connectée avec activation softmax pour la classification
        layers.Dense(7, activation='softmax', kernel_initializer=Orthogonal())
    ])
    return model

In [ ]:
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2], 1)

In [ ]:
rnn_model = build_rnn_model(input_shape)

In [ ]:
rnn_model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
rnn_model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=20)

Epoch 1/20
391/391 [==============================] - 120s 275ms/step - loss: 1.0643 - accuracy: 0.6162 - val_loss: 0.5707 - val_accuracy: 0.8062
Epoch 2/20
391/391 [==============================] - 98s 251ms/step - loss: 0.4397 - accuracy: 0.8554 - val_loss: 0.3678 - val_accuracy: 0.8799
Epoch 3/20
391/391 [==============================] - 107s 274ms/step - loss: 0.3024 - accuracy: 0.9009 - val_loss: 0.3221 - val_accuracy: 0.8981
Epoch 4/20
391/391 [==============================] - 112s 286ms/step - loss: 0.2432 - accuracy: 0.9191 - val_loss: 0.2738 - val_accuracy: 0.9113
Epoch 5/20
391/391 [==============================] - 104s 266ms/step - loss: 0.1956 - accuracy: 0.9342 - val_loss: 0.2810 - val_accuracy: 0.9061
Epoch 6/20
391/391 [==============================] - 98s 251ms/step - loss: 0.1589 - accuracy: 0.9476 - val_loss: 0.2741 - val_accuracy: 0.9158
Epoch 7/20
391/391 [==============================] - 107s 274ms/step - loss: 0.1513 - accuracy: 0.9481 - val_loss: 0.2809 - v

In [ ]:
test_loss, test_acc = rnn_model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

163/163 - 9s - loss: 0.1963 - accuracy: 0.9424 - 9s/epoch - 57ms/step

Test accuracy: 0.9423520565032959


In [ ]:
tf.keras.models.save_model(rnn_model,'RNN_2.keras')